<a href="https://colab.research.google.com/github/Muhammadridho100902/google_collab/blob/main/Car_vs_Bike_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import os
from PIL import Image
import zipfile
import os
import shutil
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
! chmod 600 /content/kaggle.json
! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download -d utkarshsaxenadn/car-vs-bike-classification-dataset

 87% 89.0M/103M [00:00<00:00, 101MB/s]
100% 103M/103M [00:00<00:00, 109MB/s] 


In [4]:
dataset_zip = zipfile.ZipFile('car-vs-bike-classification-dataset.zip', 'r')
dataset_zip.extractall()
dataset_zip.close()

In [5]:
dataset_dir = 'dataset'

train_dir = os.path.join(dataset_dir, 'train') # 'dataset/train'
val_dir = os.path.join(dataset_dir, 'val') # 'dataset/train'

car_train_dir = os.path.join(train_dir, 'car')
bike_train_dir = os.path.join(train_dir, 'bike')

car_val_dir = os.path.join(val_dir, 'car')
bike_val_dir = os.path.join(val_dir, 'bike')

In [6]:
os.mkdir(dataset_dir)
os.mkdir(train_dir)
os.mkdir(val_dir)
os.mkdir(car_train_dir)
os.mkdir(bike_train_dir)
os.mkdir(car_val_dir)
os.mkdir(bike_val_dir)

In [10]:
training_proportion = 0.8
car_train_length = training_proportion * len(os.listdir("Car-Bike-Dataset/Car"))
bike_train_length = training_proportion * len(os.listdir("Car-Bike-Dataset/Bike"))

In [13]:
filecounter = 0
for file in os.listdir('Car-Bike-Dataset/Car'):
  src = os.path.join('Car-Bike-Dataset/Car', file)
  if filecounter <= car_train_length:
    dest = car_train_dir
  else:
    dest = car_val_dir

  shutil.move(src, dest)
  filecounter += 1
  print('Move {} from {} to {}'.format(file, src, dest))

print("BANYAK DATA CAR UNTUK TRAINING: {} DATA".format(len(os.listdir(car_train_dir))))
print("BANYAK DATA CAR UNTUK VALIDATION: {} DATA".format(len(os.listdir(car_val_dir))))

Move Car (336).jpeg from Car-Bike-Dataset/Car/Car (336).jpeg to dataset/train/car
Move Car (1782).jpeg from Car-Bike-Dataset/Car/Car (1782).jpeg to dataset/train/car
Move Car (1455).jpeg from Car-Bike-Dataset/Car/Car (1455).jpeg to dataset/train/car
Move Car (1606).jpeg from Car-Bike-Dataset/Car/Car (1606).jpeg to dataset/train/car
Move Car (2).jpeg from Car-Bike-Dataset/Car/Car (2).jpeg to dataset/train/car
Move Car (1644).jpeg from Car-Bike-Dataset/Car/Car (1644).jpeg to dataset/train/car
Move Car (1865).jpeg from Car-Bike-Dataset/Car/Car (1865).jpeg to dataset/train/car
Move Car (691).jpeg from Car-Bike-Dataset/Car/Car (691).jpeg to dataset/train/car
Move Car (1468).jpeg from Car-Bike-Dataset/Car/Car (1468).jpeg to dataset/train/car
Move Car (1407).jpeg from Car-Bike-Dataset/Car/Car (1407).jpeg to dataset/train/car
Move Car (1155).jpeg from Car-Bike-Dataset/Car/Car (1155).jpeg to dataset/train/car
Move Car (1474).jpeg from Car-Bike-Dataset/Car/Car (1474).jpeg to dataset/train/car
Mo

In [14]:
filecounter = 0
for file in os.listdir('Car-Bike-Dataset/Bike'):
  src = os.path.join('Car-Bike-Dataset/Bike', file)
  if filecounter <= bike_train_length:
    dest = bike_train_dir
  else:
    dest = bike_val_dir

  shutil.move(src, dest)
  filecounter += 1
  print('Move {} from {} to {}'.format(file, src, dest))

print("BANYAK DATA BIKE UNTUK TRAINING: {} DATA".format(len(os.listdir(bike_train_dir))))
print("BANYAK DATA BIKE UNTUK VALIDATION: {} DATA".format(len(os.listdir(bike_val_dir))))

Move Bike (205).jpeg from Car-Bike-Dataset/Bike/Bike (205).jpeg to dataset/train/bike
Move Bike (1005).jpeg from Car-Bike-Dataset/Bike/Bike (1005).jpeg to dataset/train/bike
Move Bike (1218).jpeg from Car-Bike-Dataset/Bike/Bike (1218).jpeg to dataset/train/bike
Move Bike (938).jpeg from Car-Bike-Dataset/Bike/Bike (938).jpeg to dataset/train/bike
Move Bike (663).jpeg from Car-Bike-Dataset/Bike/Bike (663).jpeg to dataset/train/bike
Move Bike (1188).jpeg from Car-Bike-Dataset/Bike/Bike (1188).jpeg to dataset/train/bike
Move Bike (655).jpeg from Car-Bike-Dataset/Bike/Bike (655).jpeg to dataset/train/bike
Move Bike (1400).jpeg from Car-Bike-Dataset/Bike/Bike (1400).jpeg to dataset/train/bike
Move image (23).jpeg from Car-Bike-Dataset/Bike/image (23).jpeg to dataset/train/bike
Move Bike (643).jpeg from Car-Bike-Dataset/Bike/Bike (643).jpeg to dataset/train/bike
Move Bike (533).jpeg from Car-Bike-Dataset/Bike/Bike (533).jpeg to dataset/train/bike
Move Bike (826).jpeg from Car-Bike-Dataset/Bik

In [15]:
training = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    vertical_flip = True
)

val = ImageDataGenerator(
    rescale = 1./255,
)

In [16]:
train_generator = training.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary', # Karena hanya memiliki 2 kela yaitu bike dan car, jika lebih itu disebut categorical
    shuffle = True
)

val_generator = val.flow_from_directory(
    val_dir,
    target_size = (150, 150),
    batch_size = 20,
    class_mode = 'binary',
    shuffle = True
)

Found 3202 images belonging to 2 classes.
Found 798 images belonging to 2 classes.


In [17]:
train_generator.class_indices

{'bike': 0, 'car': 1}

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (3,3), activation = 'relu', input_shape=(150,150,3)), #150,150 mengikuti target size
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(), # agar menghasilkan array 1 dimensi seperti ([0.1, 0.2, 0.3])

    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 148, 148, 8)       224       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 74, 74, 8)         0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 43808)             0         
                                                                 
 dense (Dense)               (None, 8)                 350472    
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 350705 (1.34 MB)
Trainable params: 350705 (1.34 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [24]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['acc']
)

In [25]:
model.fit(
    train_generator,
    validation_data = val_generator,
    epochs = 5
)

Epoch 1/5
 42/161 [======>.......................] - ETA: 26s - loss: 1.0508 - acc: 0.5633

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


161/161 [==============================] - 39s 239ms/step - loss: 0.6555 - acc: 0.7011 - val_loss: 0.4941 - val_acc: 0.7644
Epoch 2/5
161/161 [==============================] - 40s 245ms/step - loss: 0.4066 - acc: 0.8210 - val_loss: 0.3941 - val_acc: 0.8296
Epoch 3/5
161/161 [==============================] - 38s 234ms/step - loss: 0.3521 - acc: 0.8379 - val_loss: 0.3677 - val_acc: 0.8471
Epoch 4/5
161/161 [==============================] - 38s 235ms/step - loss: 0.3509 - acc: 0.8504 - val_loss: 0.3598 - val_acc: 0.8434
Epoch 5/5
161/161 [==============================] - 43s 267ms/step - loss: 0.2727 - acc: 0.8873 - val_loss: 0.3302 - val_acc: 0.8659


untuk meng handle error pada saat fit model

In [26]:
import PIL
from PIL import Image

for file in os.listdir(car_train_dir):
  img = Image.open(os.path.join(car_train_dir, file))

  if img.format == 'PNG' and img.mode != 'RGBA':
    img.convert('RGBA').save(os.path.join(car_train_dir, file))

In [27]:
for file in os.listdir(bike_train_dir):
  img = Image.open(os.path.join(bike_train_dir, file))

  if img.format == 'PNG' and img.mode != 'RGBA':
    img.convert('RGBA').save(os.path.join(bike_train_dir, file))

In [28]:
for file in os.listdir(car_val_dir):
  img = Image.open(os.path.join(car_val_dir, file))

  if img.format == 'PNG' and img.mode != 'RGBA':
    img.convert('RGBA').save(os.path.join(car_val_dir, file))

In [29]:
for file in os.listdir(bike_val_dir):
  img = Image.open(os.path.join(bike_val_dir, file))

  if img.format == 'PNG' and img.mode != 'RGBA':
    img.convert('RGBA').save(os.path.join(bike_val_dir, file))

In [30]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
my_model = tf.keras.models.load_model('model.h5')

In [ ]:
my_model.get_config()

In [41]:
def predict_image(fileimage):
  image_file = fileimage

  image = tf.keras.utils.load_img(image_file, target_size = (150,150))

  image_array = tf.keras.utils.img_to_array(image)

  image_array = image_array / 255. # untuk menghasilkan array dengan nilai antara 0 - 1

  image_array = np.array([image_array])

  predict_result = my_model.predict(image_array)

  predict_result = predict_result[0][0]

  if predict_result < 0.5:
    print("MOTOR")
  else:
    print("MOBIL")

In [45]:
predict_image('OIP.jpeg')

1/1 [==============================] - 0s 27ms/step
MOTOR
